In [1]:
import crypten
import torch
import sys
import os
from crypten.config import cfg
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path

sys.path.append(parent_dir)


from privacy_utils import *
import pandas as pd

crypten.init()

/Users/jake/miniconda3/envs/Crypten/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crypten.cfg.config

{'communicator': {'verbose': False}, 'debug': {'debug_mode': False, 'validation_mode': False}, 'encoder': {'precision_bits': 16}, 'functions': {'max_method': 'log_reduction', 'exp_iterations': 8, 'reciprocal_method': 'NR', 'reciprocal_nr_iters': 10, 'reciprocal_log_iters': 1, 'reciprocal_all_pos': False, 'reciprocal_initial': None, 'sqrt_nr_iters': 3, 'sqrt_nr_initial': None, 'sigmoid_tanh_method': 'reciprocal', 'sigmoid_tanh_terms': 32, 'log_iterations': 2, 'log_exp_iterations': 8, 'log_order': 8, 'trig_iterations': 10, 'erf_iterations': 8}, 'mpc': {'active_security': False, 'provider': 'TFP', 'protocol': 'beaver'}, 'nn': {'dpsmpc': {'protocol': 'layer_estimation', 'skip_loss_forward': True, 'cache_pred_size': True}}}

In [3]:
# crypten.cfg.load_config("/Users/jake/Python/causal_prospective_merge/crypten_cfg.yaml")

Implementing linear EIG calc in crypten

In [4]:
n_host_sample = 2000 
sigma_error = 1
d = 10
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [5]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.std()) * (Y_host-Y_host.mean()) + sigma_error * torch.randn_like(Y_host)

In [6]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [7]:
sigma_error = 1
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)

In [8]:
X_host_no_T_2 = (torch.randn((n_host_sample,d)) @ A ) 
T_host_2 = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T_2 = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host_2 = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [9]:
compare_obs_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),DP=True,epsilon=10)

{'EIG_obs_torch': 0.3996301293373108,
 'EIG_obs_crypten': 0.4208526611328125,
 'EIG_obs_DP': 0.0499675550797204}

In [10]:
compare_caus_EIG_lin(X_host,X_host,torch.eye(X_host.shape[1]),d,DP=True,epsilon=10)

{'EIG_caus_torch': 0.1392507255077362,
 'EIG_caus_crypten': 0.1504364013671875,
 'EIG_caus_DP': 5.214573520818256}

In [11]:
def bairess_alg(encrypt,decrypted):
    M = encrypt # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in tqdm(range(N-1)):
        if decrypted[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if decrypted[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in (range(i+1,N)):
            for k in range(i+1,N):
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ).div(prev)
        prev = M[i][i]
    return sign * M[-1][-1]

In [12]:
XX_host = X_host.T @ X_host + np.eye(X_host.shape[1])
XX_host_crypt = crypten.mpc.MPCTensor(XX_host)

In [13]:
a = bairess_alg(XX_host_crypt,XX_host)

100%|██████████| 19/19 [01:05<00:00,  3.43s/it]


In [14]:
torch.log(a.get_plain_text())

tensor(0.4328)

In [15]:
a.get_plain_text()

tensor(1.5416)

## IDHP

In [16]:
n_host = 1000
exp_parameters = {'number_of_candidate_sites': 20+1,
                'min_sample_size_cand': 200, 
                'max_sample_size_cand': 400, 
                'host_sample_size': n_host,
                'host_test_size': 2000,
                'outcome_function': None, 
                'std_true_y': 1, 
                'power_x': 1, 
                'power_x_t': 1}
epsilon = 10

In [17]:
data_with_groundtruth, x, t, y = get_data("IDHP", "")
XandT = pd.concat([x,t], axis=1)
XandT = XandT.sample(n=10**4, replace=True, random_state=42)


candidate_sites = generating_random_sites_from(XandT, data_with_groundtruth, exp_parameters, added_T_coef=40, binary_outcome=False)  

host = candidate_sites[0][:n_host]
XandT_host, Y_host = torch.from_numpy(host.drop(columns=["Y"]).values), torch.from_numpy(host["Y"].values)
host_test = candidate_sites[0][n_host:]
causal_param_first_index = np.shape(XandT)[1]

candidate_sites = {key: value for key, value in candidate_sites.items() if key != 0}

cov = torch.eye(XandT_host.shape[1])

/Users/jake/Python/causal_prospective_merge/privacy_utils.py:268: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [18]:
# privacy_results_index = {
#     'EIG_obs_torch':[],
#     'EIG_obs_crypten':[],
#     'EIG_obs_DP' : [],
#     'EIG_caus_torch':[],
#     'EIG_caus_crypten':[],
#     'EIG_caus_DP':[]
# }

# for _, candidate in tqdm(candidate_sites.items()):
#     X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
#     obs_results_dict = compare_obs_EIG_lin(XandT_host,X_cand,cov,DP=True,epsilon=epsilon)
#     caus_results_dict = compare_caus_EIG_lin(XandT_host,X_cand,cov,DP=True,epsilon=epsilon)

#     for key in obs_results_dict:
#         privacy_results_index[key].append( obs_results_dict[key] )
    
#     for key in caus_results_dict:
#         privacy_results_index[key].append( caus_results_dict[key] )

In [19]:
# privacy_results_index

In [20]:
X_cand = torch.from_numpy(candidate_sites[1].drop(columns=["Y"]).values)
X_host = XandT_host

In [21]:
XX_cand = X_cand.T @ X_cand
X_host_plus_cov = X_host.T @ X_host + cov
X_host_plus_cov_inv = torch.linalg.inv(X_host_plus_cov)
_,EIG_torch = torch.slogdet((XX_cand @ X_host_plus_cov_inv + torch.eye(XX_cand.shape[0])))

X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov_inv)
X_cand_crypten = crypten.mpc.MPCTensor(XX_cand)
# EIG_crypten = logdet_Crypten(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0])).get_plain_text()

In [22]:
EIG_torch

tensor(11.0839, dtype=torch.float64)

In [23]:
((XX_cand @ X_host_plus_cov_inv - (X_cand_crypten @ X_host_crypten).get_plain_text())**2).mean()

tensor(4.4224e-06, dtype=torch.float64)

In [24]:
logdet_Crypten((X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]))).get_plain_text()

tensor(2.7406e+09)

In [102]:
L,D = chol_LD_Crypten(((X_cand_crypten + X_host_crypten)/len(X_host_crypten)))
c = D.get_plain_text()

In [104]:
D_log_enc = D.log()
D_log_enc.sum().get_plain_text()

tensor(-66.0366)

In [100]:
np.log(c).sum() + len(c)*np.log(len(X_host_crypten))

tensor(139.9559)

In [99]:
len(XX_cand)

52

In [105]:
torch.slogdet((XX_cand + X_host_plus_cov_inv)/len(XX_cand)) 

torch.return_types.slogdet(
sign=tensor(1., dtype=torch.float64),
logabsdet=tensor(-65.4928, dtype=torch.float64))

In [90]:
b = torch.linalg.ldl_factor((XX_cand + X_host_plus_cov_inv) /len(X_host_plus_cov_inv))[0].diag()

In [87]:
np.log(b).sum()+len(b)*np.log(100)

tensor(173.9760, dtype=torch.float64)

In [64]:
torch.log(torch.abs(a)[0].diag())

TypeError: abs(): argument 'input' (position 1) must be Tensor, not torch.return_types.linalg_ldl_factor

In [66]:
torch.linalg.cholesky(XX_cand + X_host_plus_cov_inv +torch.eye(XX_cand.shape[0]))

tensor([[14.7693,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.3290, 14.8053,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6013, 12.2576,  7.8191,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.1361,  0.1618, -0.0268,  ...,  1.4702,  0.0000,  0.0000],
        [ 0.2039,  0.0804, -0.0923,  ..., -0.0915,  1.6310,  0.0000],
        [ 1.8290,  0.0403,  0.1559,  ..., -0.2588, -0.1564,  2.6115]],
       dtype=torch.float64)

In [67]:
torch.prod(a[0].diag())

tensor(-30831.2644, dtype=torch.float64)

In [35]:
def choleski(a):
    n = len(a)
    for k in range(n):
        a[k,k] = (a[k,k] -  a[k,0:k] @ a[k,0:k]).sqrt()
        for i in range(k+1,n):
            a[i,k] = (a[i,k] - a[i,0:k] @ a[k,0:k])/a[k,k]
    for k in range(1,n): a[0:k,k] = 0.0
    return a

In [37]:
a = choleski(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]))

In [ ]:
a_correct = choleski(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]))

In [40]:
plain_text = a.get_plain_text()

In [41]:
torch.diag(plain_text)

tensor([ 1.3731e+00,  1.0809e+00,  1.1151e+00,  1.0815e+00,  1.0622e+00,
         1.0787e+00,  8.6667e-01,  1.0089e+00,  1.0988e+00,  9.0668e-01,
         1.0940e+00,  1.1447e+00,  1.1266e+00,  1.0273e+00, -2.0645e+09,
         1.9799e+09, -2.3924e+08, -7.5471e+08, -1.2316e+09,  1.2787e+09,
        -7.4276e+08, -1.7109e+09, -2.0575e+09, -1.2524e+09,  2.3476e+08,
        -3.9914e+08, -2.7453e+08,  6.4315e+07, -2.7571e+08, -1.7941e+09,
        -5.8454e+08, -1.8026e+09, -2.0025e+09,  2.7680e+08, -1.0890e+09,
        -9.9119e+08,  1.2683e+09,  5.8616e+08, -1.1281e+09,  4.5381e+08,
         1.2214e+09, -1.5685e+09,  2.1241e+09, -1.5466e+09, -1.7308e+09,
        -4.6608e+08, -6.3313e+08, -5.2956e+08, -5.5944e+08, -5.4547e+08,
         1.4762e+09,  1.7846e+09])

In [34]:
X_cand_crypten.sq

tensor([ 1.8814,  1.1676,  1.2472,  1.1714,  1.1313,  1.1658,  0.7538,  1.0200,
         1.2116,  0.8259,  1.2012,  1.3140,  1.2728,  1.0577, -2.5659,  1.4263,
         1.3614,  1.7462,  1.2629,  1.2134,  1.1926,  1.2353,  1.0685,  1.2052,
         1.0732,  1.2899,  0.8869,  1.2257,  1.2417,  1.2375,  1.3422,  1.2943,
         1.2717,  1.2162,  1.2337,  1.1995,  1.2212,  1.2369,  1.2307,  1.1978,
         1.1960,  1.2608,  1.1825,  1.1426,  1.1508,  1.1650,  1.2741,  1.2078,
         1.1424,  1.2924,  1.1560,  1.1995])

In [33]:
((D.get_plain_text()-D_true)**2).mean()

tensor(1.9748e-05)

In [26]:
L_true,D_true = chol_LD_torch(XX_cand @ X_host_plus_cov_inv +torch.eye(XX_cand.shape[0]))

In [27]:
L,D = chol_LD_Crypten((X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0])))
D.get_plain_text()

KeyboardInterrupt: 

In [ ]:
a = bairess_alg(X_cand_crypten @ X_host_crypten + torch.eye(XX_cand.shape[0]),(XX_cand @ X_host_plus_cov_inv + torch.eye(XX_cand.shape[0])))
a

100%|██████████| 51/51 [22:10<00:00, 26.08s/it]


MPCTensor(
	_tensor=73350087966301
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)

In [ ]:
torch.log(a.get_plain_text())

tensor(20.8359)

In [ ]:
(a.log()).get_plain_text()

tensor(2.5102e+10)

In [ ]:
def householder(a):
    n = len(a)
    for k in range(n-2):
        u = a[k+1:n,k]
        uMag = math.sqrt(np.dot(u,u))
        if u[0] < 0.0: uMag = -uMag
        u[0] = u[0] + uMag
        h = np.dot(u,u)/2.0
        v = np.dot(a[k+1:n,k+1:n],u)/h
        g = np.dot(u,v)/(2.0*h)
        v = v - g*u
        a[k+1:n,k+1:n] = a[k+1:n,k+1:n] - np.outer(v,u)  \
                         -np.outer(u,v)
    return np.diagonal(a),np.diagonal(a,1)

In [ ]:
torch.log(a)

KeyboardInterrupt: 

In [ ]:
XX_cand
X_cand_crypten

MPCTensor(
	_tensor=tensor([[13500416,   790528, -1135005,  ...,   131072,    65536,   786432],
        [  790528, 12845925,  9974298,  ...,   177611,    51948,    30833],
        [-1135005,  9974298, 11636678,  ...,   130680,    39123,     5836],
        ...,
        [  131072,   177611,   130680,  ...,   131072,        0,        0],
        [   65536,    51948,    39123,  ...,        0,    65536,        0],
        [  786432,    30833,     5836,  ...,        0,        0,   786432]])
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)

KeyboardInterrupt: 

In [ ]:
L.get_plain_text().min()

tensor(-18.0751)

In [ ]:
d_plain = D.get_plain_text()

In [ ]:
torch.abs(d_plain)

tensor([ 5.6834,  7.4158, -0.2931, 26.2756,  1.4323,  2.0508,  2.2263,  2.4757,
         2.1785,  2.3619,  2.5277,  0.4344, -1.3256, 20.7987, 26.1783,  1.2382,
         0.3080, -2.7296,  8.1274,  1.6298,  2.5500,  0.7549,  2.6070,  1.8040,
         1.6967,  1.1568,  0.9444,  1.0805,  1.0641,  1.0747,  1.0839,  1.0892,
         1.1012,  1.0755,  1.1101,  1.0678,  1.0797,  1.0913,  1.0876,  1.0640,
         1.1266,  1.0649,  1.0638,  1.0567,  1.0213,  1.0413,  1.0454,  1.0862,
         1.0321,  1.2468,  1.0170,  1.0797])

In [ ]:
L.get_plain_text()

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 4.3465e-01,  1.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.5124e-01,  5.5630e-01,  1.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-1.9073e-03,  6.5613e-04, -4.5166e-03,  ...,  1.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.8311e-04, -7.6294e-05,  7.7820e-04,  ..., -1.6479e-03,
          1.0000e+00,  0.0000e+00],
        [-5.2643e-03,  1.6479e-03, -4.6844e-03,  ..., -1.8845e-02,
         -7.0801e-03,  1.0000e+00]])

In [ ]:
torch.linalg.lu_factor(XX_cand @ X_host_plus_cov_inv + torch.eye(XX_cand.shape[0]))

torch.return_types.linalg_lu_factor(
LU=tensor([[ 5.6900e+00,  1.5797e+01,  7.4488e-01,  ..., -5.2595e-01,
          2.4484e+00,  5.8242e-01],
        [ 9.9708e-01, -5.0926e+00, -5.2313e-01,  ...,  2.6421e-01,
         -7.7259e-01, -2.3422e-01],
        [ 2.5166e-01, -1.5067e-01,  2.0935e+00,  ..., -4.6215e-01,
         -3.4607e-01,  3.5035e-02],
        ...,
        [-1.8953e-03, -5.9370e-03, -1.9955e-04,  ...,  1.2441e+00,
         -1.3452e-02, -8.8851e-03],
        [ 1.9717e-04,  6.4983e-04, -3.1833e-05,  ..., -2.0259e-03,
          1.0179e+00,  1.1374e-03],
        [-5.2064e-03, -1.5931e-02, -1.9270e-03,  ..., -2.3199e-02,
         -4.9377e-03,  1.0888e+00]], dtype=torch.float64),
pivots=tensor([15, 15,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52],
       dtype=torch.int32))

In [ ]:
with crypten.mpc.ConfigManager("reciprocal_nr_iters", 100):
    XX_cand_caus = X_cand.T @ X_cand
    X_host_plus_cov = X_host.T @ X_host + cov
    XX_cand_causal,X_host_plus_cov = XX_cand_caus[causal_param_first_index:,causal_param_first_index:],X_host_plus_cov[causal_param_first_index:,causal_param_first_index:]
    X_host_plus_cov_inv_caus = torch.linalg.inv(X_host_plus_cov)
    _,EIG_torch_caus = torch.slogdet(XX_cand_causal @ X_host_plus_cov_inv_caus + torch.eye(X_host_plus_cov_inv_caus.shape[0]))

    X_host_crypten = crypten.mpc.MPCTensor(X_host_plus_cov_inv)
    XX_cand_crypten = crypten.mpc.MPCTensor(XX_cand_causal)
    EIG_crypten_caus = logdet_Crypten(XX_cand_crypten @ X_host_crypten + torch.eye(X_host_plus_cov_inv_caus.shape[0])).get_plain_text()

AttributeError: module 'crypten.mpc' has no attribute 'ConfigManager'

In [ ]:
print(EIG_torch_caus,EIG_crypten_caus)

tensor(12.4140, dtype=torch.float64) tensor(11.0452)


In [ ]:
D_plain = D.get_plain_text()

In [ ]:
torch.log(torch.abs(D_plain)).sum()

tensor(20.7083)

In [ ]:
torch.log(D_plain).sum()

tensor(nan)

## Misc

In [ ]:
# if type(X_host) is torch.Tensor:
# 
cov = np.eye(X_host.shape[1])
y = np.zeros(len(X_host))

Z = get_diff_private_version(X_host,y)

XX_host = Z["XX"]

_,logdet_1 = np.linalg.slogdet(X_cand.T @ X_cand + XX_host + cov)
_,logdet_2 = np.linalg.slogdet(XX_host + cov)

RuntimeError: 1D tensors expected, but got 2D and 2D tensors

In [ ]:
X = np.array(X_host)
y = np.array(Y_host)

In [ ]:
y .dot(y)

4069.0527

In [ ]:
# sufficient statistics
S = {'XX': X.T.dot(X),
        'Xy': X.T.dot(y),
        'yy': y .dot(y)
        }

posteriors = {}

# if 'non-private' in methods:
#     posteriors['non-private'] = run_non_private(model_prior_params, S, N)


#posteriors['naive'] = run_naive(model_prior_params, S, N, sensitivity_x, sensitivity_y, epsilon)

In [ ]:
X = np.array(X_host)
y = np.array(Y_host)

sensitivity_x, sensitivity_y = get_sensativitiy(X,y)

epsilon = 1

In [ ]:
data_dim = S['XX'].shape[0]

XX_comps = data_dim * (data_dim + 1) / 2  # upper triangular, not counting last column which is X
X_comps = data_dim  # last column
Xy_comps = data_dim
yy_comps = 1
sensitivity = XX_comps * sum(sensitivity_x[:-1]) ** 2 \
                + X_comps * sum(sensitivity_x[:-1]) \
                + Xy_comps * sum(sensitivity_x[:-1]) * sensitivity_y \
                + yy_comps * sensitivity_y ** 2

Z = {key: np.random.laplace(loc=val, scale=sensitivity / epsilon) for key, val in S.items()}

# symmetrize Z_XX since we only want to add noise to upper triangle
Z['XX'] = symmetrize(Z['XX'])

Z['X'] = Z['XX'][:, 0][:, None]

In [ ]:
Z["X"].mean()

2205.008211286303

In [ ]:
X_host.mean()

-8.1423976e-05

In [ ]:
Z.keys()

dict_keys(['XX', 'Xy', 'yy', 'X'])

In [ ]:
priv_XX = torch.tensor(Z["XX"])

In [ ]:
(X_host.T @ X_host).mean()

0.018392561

In [ ]:
type(X_host)

numpy.ndarray

In [ ]:
priv_XX.mean()

tensor(455.8781, dtype=torch.float64)

In [ ]:
np.linalg.slogdet(priv_XX +np.eye(X_host.shape[1]) )

(-1.0, 186.1891397270062)

In [ ]:
torch.linalg.slogdet(X_host.T @ X_host +np.eye(X_host.shape[1]))

TypeError: linalg_slogdet(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
priv_X - X_host.T @ X_host

tensor([[ 4.1798, -0.1939,  2.3292,  5.1394, -4.4555,  1.1445,  1.1193, -4.0168,
          1.9040,  2.1432,  2.0106, -0.1350,  1.0632,  2.4952, -2.1895,  0.5696,
          0.4913, -1.8778,  0.8573,  0.9238],
        [-0.1939,  7.7291, -3.9290, -1.8749,  2.1200, -0.8760,  2.3354, -1.7791,
         -4.5772, -2.4300, -0.1276,  3.8359, -1.8889, -0.8781,  1.1507, -0.5232,
          1.0409, -0.9258, -2.3725, -1.2714],
        [ 2.3292, -3.9290,  6.9166,  4.7498,  0.2810,  1.2035, -3.9490,  0.9760,
          2.9695,  2.4714,  1.0389, -1.9428,  3.3068,  2.2264,  0.1712,  0.6588,
         -2.0021,  0.6660,  1.3276,  1.1224],
        [ 5.1394, -1.8749,  4.7498,  8.8634, -5.6383,  1.4736,  0.3408, -4.7306,
          4.0217,  3.2184,  2.4766, -0.9318,  2.1888,  4.2901, -2.7647,  0.7033,
          0.1061, -2.2913,  1.8564,  1.4242],
        [-4.4555,  2.1200,  0.2810, -5.6383,  7.8555, -3.6295, -2.4313,  2.7517,
         -3.7156, -4.4249, -2.1930,  1.1223,  0.1595, -2.7562,  3.9227, -1.7315,
      

In [ ]:
X[:,0].max()

0.061485384

In [ ]:
X[:,0].min()

-0.059155844

In [ ]:
def get_sensativitiy(X,y):

    sens_x = np.zeros(X.shape[1])

    for i in range((X.shape[1])):
        sens_x[i] = X[:,i].max() - X[:,i].min()
    
    sens_y = y.max() - y.min()

    return sens_x,sens_y